In [1]:
import traceback

API ENDPOINT SCRAPE

In [2]:
import time

In [3]:
from requests_module import Request

In [4]:
import requests

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('Ecomm Data.csv')

In [7]:
df.iloc[5359]

Unnamed: 0                                                            5359
name                                                          Carolyn Keys
token                                                         b_exb2xs3ag9
urls                     https://www.faire.com/api/brand-view/b_exb2xs3ag9
owner_user_token                                              u_29lk17tycj
short_description        Carolyn Keys is a line of handmade modern jewe...
description              Carolyn Keys is a line of handmade modern jewe...
url                                                http://carolynkeys.com/
instagram_handle                                        carolynkeysjewelry
categories                                                             NaN
deranked_categories                                                    NaN
made_in                                                                USA
based_in                                                               USA
based_in_state           

In [8]:
url = 'https://www.faire.com/api/brand-view/b_dd0d5049'

In [9]:
json = requests.get(url).json()

In [11]:
brand_keys = list(json['brand'].keys())
brand_keys.sort()

In [12]:
'description' in brand_keys

True

In [13]:
for key in K:
    print(key)

NameError: name 'K' is not defined

In [14]:
json['brand']['brand_reviews_summary']['average_rating']

4.9

In [15]:
json['brand']['brand_reviews_summary']['number_of_reviews']

35

In [16]:
json['related_search_terms']

['diy+kit', 'manicure kit']

In [17]:
found = list()
not_found = list()
for col in df.columns:
    if col in json['brand']:
        found.append(col)
        
    else:
        not_found.append(col)
        
for item in found:
    print('Column:', item, ' found in brand')
    
print('---------------------------')

for item in not_found:
    print('Column:', item, 'not found')

Column: name  found in brand
Column: token  found in brand
Column: owner_user_token  found in brand
Column: short_description  found in brand
Column: description  found in brand
Column: url  found in brand
Column: instagram_handle  found in brand
Column: categories  found in brand
Column: deranked_categories  found in brand
Column: made_in  found in brand
Column: based_in  found in brand
Column: based_in_state  found in brand
Column: based_in_city  found in brand
Column: ships_from_country  found in brand
Column: creation_year  found in brand
Column: hand_made  found in brand
Column: lead_time_days  found in brand
Column: sold_on_amazon  found in brand
Column: active_products_count  found in brand
Column: facebook_followers  found in brand
Column: instagram_followers  found in brand
Column: twitter_followers  found in brand
Column: last_product_added_at  found in brand
Column: owner_first_name  found in brand
Column: owner_last_name  found in brand
Column: minimum_order_amount  found i

In [18]:
for col in df.columns:
    print(col)

Unnamed: 0
name
token
urls
owner_user_token
short_description
description
url
instagram_handle
categories
deranked_categories
made_in
based_in
based_in_state
based_in_city
ships_from_country
creation_year
hand_made
lead_time_days
sold_on_amazon
active_products_count
facebook_followers
instagram_followers
twitter_followers
last_product_added_at
average_rating
number_of_reviews
owner_first_name
owner_last_name
minimum_order_amount
primary_category
related_search_terms
is_shopify


# creating functions

In [10]:
READ_FILE = 'Ecomm Data.csv'

In [11]:
def get_similar_columns(df, json):
#     json = requests.get(url).json()
    found = list()
    for col in df.columns:
        if col in json['brand']:
            found.append(col)

    return found

In [12]:
def get_data(json):
    D = dict()
    similar_cols = get_similar_columns(df, json)
    for col in similar_cols:
        D[col] = json['brand'][col]
#     D['average_rating'] = json['
    D['average_rating'] = json['brand']['brand_reviews_summary']['average_rating']
    D['number_of_reviews'] = json['brand']['brand_reviews_summary']['number_of_reviews']
    D['related_search_terms'] = '; '.join(json['related_search_terms'])
    try:
        url = json['brand']['url']
        shopify_url = url + 'products.json'
        print('Getting shopify url: ', shopify_url)
        resp = Request.get(shopify_url)
        print('Got shopify url')
        json2 = resp.json()
        if 'products' in json2.keys():
            D['is_shopify'] = 'true'
        else:
            D['is_shopify'] = 'false'
        
    except:
        traceback.print_exc()
        D['is_shopify'] = 'false'
    finally:
        print('Value of shopify_url is: ', D['is_shopify'])
    
    return D

In [13]:
def main(start_index = None, end_index = None, ):
    try:
        df = pd.read_csv(READ_FILE)    
    except:
        df = pd.read_excel(READ_FILE)
    if start_index == None:
        start_index = 0
        
    if end_index == None:
        end_index = len(df) - 1
    
    
    for i in range(start_index, end_index + 1):
        print('--------------------------')
        print('Scraping: ', i)
        try:
            row = df.iloc[i]
            url = row['urls']
            print('Making request')
            resp = Request.get(url)
            print('Request made')
            print('Getting json')
            json = resp.json()
            print('Got json')
            unclean_json = get_data(json)
            clean_json = clean_dict(unclean_json)
            print('Cleaned JSON')
            for key in clean_json.keys():
                if key in df.columns:
                    df.loc[i, key] = clean_json[key]
            print('Added keys to CSV')
        
        
        except:
            print('Problem in Main')
            print(row['urls'])
            traceback.print_exc()
            
        finally:
            print('Sleeping')
            time.sleep(.2)
            
            
        if i % 10 ==0 and i != 0:
            try:
                df.to_csv(READ_FILE, index = False)
                print('Saving to CSV')
                    
            except:
                traceback.print_exc()
                
            
            finally:
                print(i, 'Rows Scraped.')
                    

In [14]:
def clean_dict(d):
    pass
    for key in d.keys():
        if type(d[key]) == list:
            d[key] = ', '.join(d[key])
    
    if 'minimum_order_amount' in d.keys() and type(d['minimum_order_amount']) == dict:
        d['minimum_order_amount'] = str(d['minimum_order_amount']['amount_cents']) + ' ' + str(d['minimum_order_amount']['currency'])
    return d

In [ ]:
main(14501, 16000)

--------------------------
Scraping:  14501
Making request
Request made
Getting json
Got json
Getting shopify url:  https://builtbybees.com/products.json
Value of shopify_url is:  false
Cleaned JSON
Added keys to CSV
Sleeping


Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 381, in _make_request
    self._validate_conn(conn)
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib

--------------------------
Scraping:  14502
Making request
Request made
Getting json
Got json
Getting shopify url:  http://shopbambooyou.com/products.json
Got shopify url
Value of shopify_url is:  true
Cleaned JSON
Added keys to CSV
Sleeping
--------------------------
Scraping:  14503
Making request
Request made
Getting json
Got json
Getting shopify url:  http://brainsandbeautydolls.com/products.json


Traceback (most recent call last):
  File "<ipython-input-12-38054f6dca81>", line 14, in get_data
    resp = Request.get(shopify_url)
  File "C:\Users\Lenovo\jupyter notebooks\Noah\requests_module.py", line 28, in get
    raise Exception(f'Invalid Response. Status code: {resp.status_code}')
Exception: Invalid Response. Status code: 404


Value of shopify_url is:  false
Cleaned JSON
Added keys to CSV
Sleeping
--------------------------
Scraping:  14504
Making request
Request made
Getting json
Got json
Getting shopify url:  https://naturalezasapothecary.com/products.json
Value of shopify_url is:  false
Cleaned JSON
Added keys to CSV
Sleeping


...

--------------------------
Scraping:  14957
Making request
Request made
Getting json
Got json
Getting shopify url:  http://www.boathousemercantile.com/products.json
Value of shopify_url is:  false
Cleaned JSON
Added keys to CSV
Sleeping


Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 392, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\ProgramData\Anaconda3\li

--------------------------
Scraping:  14958
Making request
Request made
Getting json
Got json
Getting shopify url:  http://mosscreekwoolworks.com/products.json
Got shopify url
Value of shopify_url is:  false
Cleaned JSON
Added keys to CSV
Sleeping


Traceback (most recent call last):
  File "<ipython-input-12-38054f6dca81>", line 17, in get_data
    if 'products' in json2.keys():
AttributeError: 'list' object has no attribute 'keys'


--------------------------
Scraping:  14959
Making request
Request made
Getting json
Got json
Getting shopify url:  http://www.dripdash.com/products.json


In [73]:
json2= Request.get(get_data(json)['url'] + 'products.json').json()

In [58]:
json2.keys()

dict_keys(['products'])

In [40]:
'; '.join(json['related_search_terms'])

'diy+kit; manicure kit'

 URLS are already filled in
 
 Average rating is at json[brand_review_summary]
 
 Number of reviews also
 
 Related search term is a key
 
 Is shopify to be done

In [20]:
get_data(json)

{'name': 'Craft a Brew',
 'token': 'b_dd0d5049',
 'owner_user_token': 'u_20i7hxhlqq',
 'short_description': 'Here at Craft a Brew we’re on a mission to turn craft beer lovers into craft beer brewers with our premium homebrew kits.',
 'description': 'Here at Craft a Brew we’re on a mission to turn craft beer lovers into craft beer brewers with our premium homebrew kits. Designed to help first-timers and hobbyists alike make the best possible beer in their very own kitchens, right out of the box. Since 2010, we’ve been perfecting our easy-to-use kits, sourcing the freshest ingredients from the same suppliers of the American craft beer industry and proudly assembling each kit by hand, under one roof in Orlando, FL.',
 'url': 'https://craftabrew.com/',
 'instagram_handle': 'craftabrew',
 'categories': [],
 'deranked_categories': [],
 'made_in': 'USA',
 'based_in': 'USA',
 'based_in_state': 'FL',
 'based_in_city': 'Orlando',
 'ships_from_country': 'USA',
 'creation_year': 2010,
 'hand_made'

In [21]:
D = get_data(json)
for key in D.keys():
    if not (type(D[key]) == str or type(D[key]) == int):
        print(key)
        print(D[key])
        print(type(D[key]))
        print('----')

categories
[]
<class 'list'>
----
deranked_categories
[]
<class 'list'>
----
hand_made
True
<class 'bool'>
----
sold_on_amazon
True
<class 'bool'>
----
minimum_order_amount
{'amount_cents': 10000, 'currency': 'USD'}
<class 'dict'>
----


Special items that are not string are : 

Categories are list

deranked_categories are list 

Minimum Order Amount is dict




In [23]:
clean_dict(D)

{'name': 'Craft a Brew',
 'token': 'b_dd0d5049',
 'owner_user_token': 'u_20i7hxhlqq',
 'short_description': 'Here at Craft a Brew we’re on a mission to turn craft beer lovers into craft beer brewers with our premium homebrew kits.',
 'description': 'Here at Craft a Brew we’re on a mission to turn craft beer lovers into craft beer brewers with our premium homebrew kits. Designed to help first-timers and hobbyists alike make the best possible beer in their very own kitchens, right out of the box. Since 2010, we’ve been perfecting our easy-to-use kits, sourcing the freshest ingredients from the same suppliers of the American craft beer industry and proudly assembling each kit by hand, under one roof in Orlando, FL.',
 'url': 'https://craftabrew.com/',
 'instagram_handle': 'craftabrew',
 'categories': '',
 'deranked_categories': '',
 'made_in': 'USA',
 'based_in': 'USA',
 'based_in_state': 'FL',
 'based_in_city': 'Orlando',
 'ships_from_country': 'USA',
 'creation_year': 2010,
 'hand_made'

In [113]:
list(df.columns)

['name',
 'token',
 'urls',
 'owner_user_token',
 'short_description',
 'description',
 'url',
 'instagram_handle',
 'categories',
 'deranked_categories',
 'made_in',
 'based_in',
 'based_in_state',
 'based_in_city',
 'ships_from_country',
 'creation_year',
 'hand_made',
 'lead_time_days',
 'sold_on_amazon',
 'active_products_count',
 'facebook_followers',
 'instagram_followers',
 'twitter_followers',
 'last_product_added_at',
 'average_rating',
 'number_of_reviews',
 'owner_first_name',
 'owner_last_name',
 'minimum_order_amount',
 'primary_category',
 'related_search_terms',
 'is_shopify']